# Forest Cover Type Classification

Import dataset

In [10]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.datasets import fetch_covtype
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# load dataset
X, y = fetch_covtype(return_X_y=True)

# create dataframe
feature_names = ['Elevation', 'Aspect', 'Slope', 'Horizontal_Distance_To_Hydrology', 'Vertical_Distance_To_Hydrology', 'Horizontal_Distance_To_Roadways', 
                 'Hillshade_9am', 'Hillshade_Noon', 'Hillshade_3pm', 'Horizontal_Distance_To_Fire_Points', 'Wilderness_Area1', 'Wilderness_Area2', 
                'Wilderness_Area3', 'Wilderness_Area4', 'Soil_Type1', 'Soil_Type2', 'Soil_Type3', 'Soil_Type4', 'Soil_Type5', 'Soil_Type6', 'Soil_Type7',
                'Soil_Type8', 'Soil_Type9', 'Soil_Type10', 'Soil_Type11', 'Soil_Type12', 'Soil_Type13', 'Soil_Type14', 'Soil_Type15', 'Soil_Type16', 
                 'Soil_Type17', 'Soil_Type18', 'Soil_Type19', 'Soil_Type20', 'Soil_Type21', 'Soil_Type22', 'Soil_Type23', 'Soil_Type24', 'Soil_Type25', 
                 'Soil_Type26', 'Soil_Type27', 'Soil_Type28', 'Soil_Type29', 'Soil_Type30', 'Soil_Type31', 'Soil_Type32', 'Soil_Type33', 'Soil_Type34', 
                 'Soil_Type35', 'Soil_Type36', 'Soil_Type37', 'Soil_Type38', 'Soil_Type39', 'Soil_Type40']

# Making a DataFrame
covtype_df = pd.DataFrame(X, columns=feature_names)
covtype_df['Cover_Type'] = y

# split into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)


# Using Neural Networks

Using the code for Project 2 (Marco Pozzoli, Lila Cassan) and the code of the course Applied Analysis and Machine Learning by Morten Hjorth-Jensen https://github.com/CompPhysics/MachineLearning/tree/master

In [11]:
#define RELU function as the activation function for the hidden layer
def relu(x):
    return np.maximum(0,x)

#define softmax as the activation function for the output layer
def softmax(a):
    expA = np.exp(a)
    return expA / expA.sum(axis=1, keepdims=True)

In [12]:
#initialization of weight and biases for a flexible number of hidden layers including the output layer
def initialize_parameters(layer_dims):
    parameters = {}
    for i in range(1, len(layer_dims)):
        parameters[f'W{i}'] = np.random.randn(layer_dims[i-1], layer_dims[i]) * 0.01
        parameters[f'b{i}'] = np.zeros((layer_dims[i], 1))
    return parameters

In [13]:
#We now write the feed forward pass
def feed_forward(X, parameters):
    cache = {'A0': X}
    for i in range(1, len(parameters)//2 + 1):
        Z = cache[f'A{i-1}'] @ parameters[f'W{i}'] + parameters[f'b{i}'].T
        A = relu(Z) if i < len(parameters)//2 else softmax(Z) #if we are at the output layer, we use softmax
        cache[f'Z{i}'] = Z
        cache[f'A{i}'] = A
    return cache

def feed_forward_hidden(X, parameters):
    cache = {'A0': X}
    for i in range(1, len(parameters)//2):  # s'arrête avant la couche de sortie
        Z = cache[f'A{i-1}'] @ parameters[f'W{i}'] + parameters[f'b{i}']
        A = relu(Z)
        cache[f'Z{i}'] = Z
        cache[f'A{i}'] = A
    return cache

In [14]:
#we now define the cost/loss function. We use the cross-entropy because it is a classification problem
def cross_entropy(prediction, target):
    epsilon = 1e-12 #added to avoid log(0)
    return -(target*np.log(prediction+epsilon) + (1-target)*np.log(1-prediction+epsilon))

def d_cross_entropy(prediction, target):   #attention: retrouver dérivée
    epsilon = 1e-12 #added to avoid log(0)
    return -(target/(prediction+epsilon) - (1-target)/(1-prediction+epsilon))

In [15]:
def backpropagation(X, Y, cache, parameters):
    m = Y.shape[0]
    gradients = {}
    output_error = d_cross_entropy(cache[f'A{len(parameters)//2}'], transformation(Y))
    for i in range(len(parameters)//2, 0, -1):
        gradients[f'dW{i}'] = 1/m * cache[f'A{i-1}'].T@ output_error
        gradients[f'db{i}'] = 1/m * np.sum(output_error, axis=0, keepdims=True)
        output_error = np.dot(output_error, parameters[f'W{i}'].T) * (cache[f'Z{i-1}'] > 0) if i > 1 else None
    return gradients

def backpropagation_update_parameters(parameters, gradients, learning_rate):
    for i in range(1, len(parameters)//2 + 1):
        parameters[f'W{i}'] -= learning_rate * gradients[f'dW{i}']
        parameters[f'b{i}'] -= learning_rate * gradients[f'db{i}'].T
    return parameters

In [26]:
#transformation
def transformation(z, num_classes = 7):
    z = np.eye(num_classes)[z-1] #because classes are from 1 to 8 and not from 0 to 7
    return z

#transformation inverse
def inverse_transformation(z):
    z = np.argmax(z, axis = 1)
    return z

In [32]:
#inizializzo dimensioni layers
n_inputs, n_features = X_train.shape
hid = 7 #number of neurons in the hidden layers
out = 7 #number of neurons in the output layer
layer_dims = [n_features, hid, out] #for one hidden layer
#layer_dims = [n_features, hid, hid, out] for two hidden layers

#inizializzo parametri
parameters = initialize_parameters(layer_dims)

#initialization of hyperparameters
learning_rate = 1e-4
epochs = 10000

#training
for i in range(epochs):
    cache = feed_forward(X_train, parameters)
    gradients = backpropagation(X_train, y_train, cache, parameters)
    parameters = backpropagation_update_parameters(parameters, gradients, learning_rate)
    if i % 100 == 0:
        print(f'Cost at iteration {i}', cross_entropy(cache[f'A{len(parameters)//2}'], transformation(y_train)).mean())

#testing
cache = feed_forward(X_test, parameters)
predictions = cache[f'A{len(parameters)//2}']
predictions = np.round(predictions)
accuracy = accuracy_score(y_test, inverse_transformation(predictions))
print('Accuracy on test set:', accuracy)

Cost at iteration 0 0.4020933762595657


/var/folders/_t/pzbtlqfs06q4gkr88b12hv9r0000gn/T/ipykernel_1373/315493064.py:7: RuntimeWarning: overflow encountered in exp
  expA = np.exp(a)
/var/folders/_t/pzbtlqfs06q4gkr88b12hv9r0000gn/T/ipykernel_1373/315493064.py:8: RuntimeWarning: invalid value encountered in divide
  return expA / expA.sum(axis=1, keepdims=True)


Cost at iteration 100 nan


KeyboardInterrupt: 